# How to match? 

## 0. Import libraries 

In [1]:
# !pip install faiss-cpu
# !pip install lightautoml

In [1]:
!python --version

Python 3.8.16


In [2]:
import pandas as pd
import warnings
import numpy as np
from lightautoml.addons.matcher import Matcher
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [5]:
# Simulating dataset with known effect size
num_users = 10000
num_months = 12

signup_months = np.random.choice(np.arange(1, num_months), num_users) * np.random.randint(0,2, size=num_users) # signup_months == 0 means customer did not sign up
df = pd.DataFrame({
    'user_id': np.repeat(np.arange(num_users), num_months),
    'signup_month': np.repeat(signup_months, num_months), # signup month == 0 means customer did not sign up
    'month': np.tile(np.arange(1, num_months+1), num_users), # months are from 1 to 12
    'spend': np.random.poisson(500, num_users*num_months) #np.random.beta(a=2, b=5, size=num_users * num_months)*1000 # centered at 500
})
# A customer is in the treatment group if and only if they signed up
df["treat"] = df["signup_month"]>0
# Simulating an effect of month (monotonically decreasing--customers buy less later in the year)
df["spend"] = df["spend"] - df["month"]*10
# Simulating a simple treatment effect of 100
after_signup = (df["signup_month"] < df["month"]) & (df["treat"])
df.loc[after_signup,"spend"] = df[after_signup]["spend"] + 100
df.head()

,user_id,signup_month,month,spend,treat
0,0,8,1,499,True
1,0,8,2,499,True
2,0,8,3,459,True
3,0,8,4,457,True
4,0,8,5,456,True


In [6]:
# Setting the signup month (for ease of analysis)
i = 3
df_i_signupmonth = (
    df[df.signup_month.isin([0, i])]
    .groupby(["user_id", "signup_month", "treat"])
    .apply(
        lambda x: pd.Series(
            {
                "pre_spends": x.loc[x.month < i, "spend"].mean(),
                "post_spends": x.loc[x.month > i, "spend"].mean(),
            }
        )
    )
    .reset_index()
)
df_i_signupmonth

,user_id,signup_month,treat,pre_spends,post_spends
0,1,0,False,484.0,417.666667
1,2,0,False,478.0,425.222222
2,3,0,False,510.0,424.222222
3,6,0,False,468.5,427.666667
4,9,0,False,464.5,412.888889
...,...,...,...,...,...
5401,9991,0,False,466.0,414.000000
5402,9992,0,False,504.0,413.222222
5403,9993,0,False,523.5,420.777778
5404,9995,0,False,496.5,414.222222


In [7]:
# Additional category features
gender = np.random.choice(a=[0,1], size=df_i_signupmonth.user_id.nunique())
age = np.random.choice(a=range(18, 70), size=df_i_signupmonth.user_id.nunique())
industry = np.random.choice(a=range(1, 3), size=df_i_signupmonth.user_id.nunique())
df_i_signupmonth['age'] = age
df_i_signupmonth['is_male'] =  gender
df_i_signupmonth['industry'] =  industry
df_i_signupmonth['industry'] = df_i_signupmonth['industry'].astype('str')
df_i_signupmonth['treat'] = df_i_signupmonth['treat'].astype(int)
df_i_signupmonth.head()

,user_id,signup_month,treat,pre_spends,post_spends,age,is_male,industry
0,1,0,0,484.0,417.666667,62,0,2
1,2,0,0,478.0,425.222222,61,0,2
2,3,0,0,510.0,424.222222,21,1,2
3,6,0,0,468.5,427.666667,38,0,2
4,9,0,0,464.5,412.888889,62,1,1


In [8]:
df_i_signupmonth.columns

Index(['user_id', 'signup_month', 'treat', 'pre_spends', 'post_spends', 'age',
       'is_male', 'industry'],
      dtype='object')

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [9]:
info_col = ['user_id', 'signup_month']

outcome = 'post_spends'
treatment = 'treat'

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [10]:
# Standard model with base parameters
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment, info_col=info_col, n_neighbors=10)
results, quality_results = model.estimate()

[20.06.2023 14:30:53 | matcher | INFO]: Applying matching


Finding index
Done
Finding index
Done


[20.06.2023 14:30:53 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: 432
[20.06.2023 14:30:53 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: 4974
[20.06.2023 14:30:53 | Faiss matcher | INFO]: Estimating quality of matching
[20.06.2023 14:30:53 | metrics | INFO]: Standardised mean difference: age           0.000312
is_male       0.000000
pre_spends    0.005606
dtype: float64
[20.06.2023 14:30:53 | metrics | INFO]: Standardised mean difference: age           0.007209
is_male       0.000000
pre_spends    0.004279
dtype: float64
[20.06.2023 14:30:53 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 0.9999999999999997, 'is_male': 1.0, 'pre_spends': 0.9999995166451553}
[20.06.2023 14:30:53 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 0.12167335017364644, 'is_male': 1.0, 'pre_spends': 0.16053502564629307}
[20.06.2023 14:30:53 | psi_pandas 

In [11]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper
ATE,100.190738,0.583088,0.0,99.047886,101.333591
ATC,100.224946,0.593753,0.0,99.061190,101.388702
ATT,99.796877,0.710921,0.0,98.403472,101.190283


### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [12]:
# group_col is used for strict comparison of categorical features.  
# In our case there is only one attribute  
# If there are several such attributes, you should make one of them and use it

group_col = "industry"

In [14]:
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment, 
                info_col=info_col, group_col=group_col)
results, quality_results = model.estimate()

[20.06.2023 14:31:06 | matcher | INFO]: Applying matching


Finding index
Done
Finding index
Done
Finding index
Done
Finding index
Done


[20.06.2023 14:31:06 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: 432
[20.06.2023 14:31:06 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: 4974
[20.06.2023 14:31:06 | Faiss matcher | INFO]: Estimating quality of matching
[20.06.2023 14:31:06 | metrics | INFO]: Standardised mean difference: age           0.079010
is_male       0.003084
pre_spends    0.044709
dtype: float64
[20.06.2023 14:31:06 | metrics | INFO]: Standardised mean difference: age           0.028257
is_male       0.032972
pre_spends    0.030633
dtype: float64
[20.06.2023 14:31:06 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 0.24874017406832236, 'is_male': 0.977378292638189, 'pre_spends': 0.7981991247404304}
[20.06.2023 14:31:06 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 0.04111527913824265, 'is_male': 0.3625816718318911, 'pre_spends': 0.00010854363642017343}
[

### 2.3 Matching but you don't know which features are better to use  
This is the easiest way to initialize and calculate metrics on a Matching task  
Use this method when you want to select the most important features and make matching based on them. 

In [16]:
model = Matcher(df=df_i_signupmonth, outcome=outcome, treatment=treatment, 
                info_col=info_col, group_col=group_col)

In [17]:
features_importance = model.lama_feature_select()
features_importance

[20.06.2023 14:31:18 | matcher | INFO]: Counting feature importance
[20.06.2023 14:31:18 | lama_feature_selector | INFO]: Getting feature scores


,Feature,Importance
0,age,35869.799805
3,industry,35869.799805
1,pre_spends,27904.770020
2,is_male,4301.930176


In [18]:
features = features_importance['Feature'].to_list()

In [19]:
# You can use both variant that you like. So you can just list the features to be matched

#results, quality_results = model.estimate(features=features_importance[:3])
results, quality_results = model.estimate(features=features[:3])

[20.06.2023 14:31:26 | matcher | INFO]: Applying matching


Finding index
Done
Finding index
Done
Finding index
Done
Finding index
Done


[20.06.2023 14:31:26 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: 432
[20.06.2023 14:31:26 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: 4974
[20.06.2023 14:31:26 | Faiss matcher | INFO]: Estimating quality of matching
[20.06.2023 14:31:26 | metrics | INFO]: Standardised mean difference: age           0.105958
is_male       0.032310
pre_spends    0.060660
dtype: float64
[20.06.2023 14:31:26 | metrics | INFO]: Standardised mean difference: age           0.055797
is_male       0.002614
pre_spends    0.038159
dtype: float64
[20.06.2023 14:31:26 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 0.02762403853104298, 'is_male': 0.21611127430192312, 'pre_spends': 0.24874017406832236}
[20.06.2023 14:31:26 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 0.00205284617918978, 'is_male': 0.9999985915838465, 'pre_spends': 5.261320715314305e-05}

In [20]:
model.matcher.df_matched

,index,pre_spends,age,is_male,industry,user_id,signup_month,pre_spends_matched,age_matched,is_male_matched,user_id_matched,signup_month_matched,industry_matched,index_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,2230,489.0,29,1,1,4193,3,463.00,24.0,1.0,4027.0,0.0,2,[5047],520.444444,424.888889,95.555556,1,0
1,2217,466.0,19,0,1,4167,3,489.50,21.0,1.0,6442.0,0.0,2,[1118],517.666667,416.111111,101.552486,1,0
2,2244,479.0,42,1,1,4228,3,463.00,30.0,1.0,4373.0,0.0,2,[320],533.888889,422.666667,111.216084,1,0
3,2262,520.5,59,1,1,4261,3,479.00,32.0,0.0,5126.0,0.0,2,[1152],513.666667,423.444444,90.238594,1,0
4,2101,518.0,55,0,1,3945,3,486.25,32.5,0.0,4776.5,0.0,1,"[1878, 106]",530.222222,427.388889,103.374920,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4969,2397,502.5,51,1,2,4489,0,469.00,25.0,1.0,7893.0,3.0,2,[3227],425.111111,522.777778,97.666667,0,1
4970,2399,491.5,57,0,2,4492,0,509.00,58.0,0.0,6529.0,0.0,2,[4817],415.333333,523.777778,109.208422,0,1
4971,2402,484.0,52,1,2,4496,0,472.50,51.0,1.0,5599.0,0.0,2,[1377],412.666667,525.555556,112.885100,0,1
4972,2337,468.5,37,1,2,4383,0,475.00,46.0,1.0,6122.0,0.0,1,[4938],419.000000,512.888889,93.120936,0,1


## 3. Results  
### 3.1 ATE, ATT, ATC

In [21]:
# model.matcher.results
results

,effect_size,std_err,p-val,ci_lower,ci_upper
ATE,100.245509,0.586057,0.0,99.096838,101.394180
ATC,100.293702,0.596747,0.0,99.124077,101.463326
ATT,99.690621,0.704271,0.0,98.310251,101.070991


### 3.2 SMD, PSI, KS-test, repeats

In [22]:
# matching quality result - SMD
model.quality_result

{'psi':                 column  anomaly_score check_result                 column  \
 0          age_treated           0.07           OK          age_untreated   
 1     industry_treated           0.00           OK     industry_untreated   
 2  post_spends_treated          16.11          NOK  post_spends_untreated   
 3   pre_spends_treated           0.04           OK   pre_spends_untreated   
 
    anomaly_score check_result  
 0           0.03           OK  
 1           0.00           OK  
 2           8.27          NOK  
 3           0.05           OK  ,
 'ks_test':             match_control_to_treat  match_treat_to_control
 age                       0.027624                0.002053
 is_male                   0.216111                0.999999
 pre_spends                0.248740                0.000053,
 'smd':             match_control_to_treat  match_treat_to_control
 age                       0.105958                0.055797
 is_male                   0.032310                0.002

In [23]:
# matching quality result - PSI
model.quality_result['psi']

,column,anomaly_score,check_result,column,anomaly_score,check_result
0,age_treated,0.07,OK,age_untreated,0.03,OK
1,industry_treated,0.00,OK,industry_untreated,0.00,OK
2,post_spends_treated,16.11,NOK,post_spends_untreated,8.27,NOK
3,pre_spends_treated,0.04,OK,pre_spends_untreated,0.05,OK


In [24]:
# matching quality result - KS-test

model.quality_result['ks_test']

,match_control_to_treat,match_treat_to_control
age,0.027624,0.002053
is_male,0.216111,0.999999
pre_spends,0.248740,0.000053


In [25]:
# matching quality result - repeats
model.quality_result['repeats']

{'match_control_to_treat': 0.93, 'match_treat_to_control': 0.08}

### 3.3 Validation
validate result with placebo treatment  
our new effect size is close to zero (it should be)

In [26]:
 
model.validate_result()
 

[20.06.2023 14:31:36 | matcher | INFO]: Applying validation of result


Finding index
Done
Finding index
Done
Finding index
Done
Finding index
Done


[20.06.2023 14:31:36 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: 432
[20.06.2023 14:31:36 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: 4974


LinAlgError: Matrix is not positive definite

In [ ]:
model.matcher.df_matched